In [1]:
import numpy as np
import scipy as sp

In [2]:
# Generate random real nxn block hermitian matrix with block size b
def randHerm(n, b = 1):
    m = n*b
    A = np.random.rand(m, m)
    for i in np.arange(m):
        for j in range(i+1, m):
            A[i, j] = A[j, i]
    return A

In [3]:
def randVec(n, b = 1):
    return np.random.rand(n*b, b)

In [116]:
def block_lanczos(H,V,k,reorth = 0):
    """
    Input
    -----
    
    H    : d x d matrix
    V    : d x b starting block
    k    : number of iterations
    
    Returns
    -------
    Q1k  : First k blocks of Lanczos vectors
    Qkp1 : final block of Lanczos vetors
    A    : diagonal blocks
    B    : off diagonal blocks (incuding block for starting with non-orthogonal block)
    """

    Z = np.copy(V)
    d,b = Z.shape
    
    A = [np.zeros((b,b),dtype=H.dtype)] * k
    B = [np.zeros((b,b),dtype=H.dtype)] * (k+1)
    
    Q = np.zeros((d,b*(k+1)),dtype=H.dtype)

    # B[0] accounts for non-orthogonal V and is not part of tridiagonal matrix
    Q[:,0:b],B[0] = np.linalg.qr(Z)
    for j in range(0,k):
        
#       Qj is the next column of blocks
        Qj = Q[:,j*b:(j+1)*b]

        if j == 0:
            Z = H@Qj
        else:
            Qjm1 = Q[:,(j-1)*b:j*b]
            Z = H @ Qj - Qjm1 @ (B[j].conj().T)
        
        # double reorthogonalization if needed
        if reorth > j:
            Z -= Q[:,:j * b] @ (Q[:,:j * b].conj().T @ Z)
            Z -= Q[:,:j * b] @ (Q[:,:j * b].conj().T @ Z)

        A[j] = Qj.conj().T @ Z
        Z -= Qj @ A[j]
        
        Q[:,(j+1)*b:(j+2)*b],B[j + 1] = np.linalg.qr(Z)
    
    Q1k = Q[:,:b*k]
    Qkp1 = Q[:,b*k:]

    return Q1k, Qkp1, A, B

In [111]:
def get_block_tridiag(A,B):
    """
    Input
    -----
    
    A  : diagonal blocks
    B  : off diagonal blocks
    
    Returns
    -------
    T  : block tridiagonal matrix
    
    """
    
    q = len(A)
    b = len(A[0])
    
    T = np.zeros((q*b,q*b),dtype=A[0].dtype)

    for k in range(q):
        T[k*b:(k+1)*b,k*b:(k+1)*b] = A[k]

    for k in range(q-1):
        T[(k+1)*b:(k+2)*b,k*b:(k+1)*b] = B[k]
        T[k*b:(k+1)*b,(k+1)*b:(k+2)*b] = B[k].conj().T
    
    return T

In [127]:
# k is number of blocks, b block size, i is the position of the identity matrix
def Ei(k, b, i):
    """
    Input
    -----
    
    k  : number of blocks
    b  : block size
    i  : position of diagonal block
    
    Returns
    -------
    Ei  : block zero vector with identity in i-th position
    
    """
        
    Ei = np.zeros((k*b,b))
    Ei[i*b:(i+1)*b,:] = np.identity(b)
    
    return Ei

In [128]:
# test output of Lanczos algorithm
n = 20
b = 2
k = 5

H = randHerm(n, b)
V = randVec(n, b)
Qk, Qkp1, M, B = block_lanczos(H, V, k, 1000)
T = get_block_tridiag(M,B[1:])

if (np.linalg.norm(Qk.T@Qk-np.identity(np.shape(Qk.T@Qk)[0])) <= 1e-8):
    print("Qk*Qk = I test passed")

if (np.linalg.norm(Qk@T + Qkp1@B[k]@Ei(k, b, k-1).conj().T - H@Qk) <= 1e-8):
    print("three term recurrence test passed")

Qk*Qk = I test passed
three term recurrence test passed
